In [2]:
import cv2
import mediapipe as mp
import math
import time
import winsound
import numpy as np

# Setup MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

# Constants
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

EAR_THRESHOLD = 0.23
DROWSINESS_BLINK_THRESHOLD = 20
GAZE_LEFT_LIMIT = 0.15
GAZE_RIGHT_LIMIT = 0.85
SUSPICIOUS_GAZE_LIMIT = 6  # max allowed gaze away events per minute

# 3D model points of facial landmarks (nose tip, chin, left eye, right eye, left mouth, right mouth)
FACE_3D_POINTS = np.array([
    (0.0, 0.0, 0.0),          # Nose tip
    (0.0, -63.6, -12.5),      # Chin
    (-43.3, 32.7, -26.0),     # Left eye left corner
    (43.3, 32.7, -26.0),      # Right eye right corner
    (-28.9, -28.9, -24.1),    # Left mouth corner
    (28.9, -28.9, -24.1)      # Right mouth corner
], dtype=np.float64)

# Corresponding 2D landmark indices in MediaPipe
FACE_2D_IDXS = [1, 152, 263, 33, 287, 57]

# Helper functions
def euclidean(p1, p2):
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])

def eye_aspect_ratio(landmarks, eye_points, w, h):
    coords = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in eye_points]
    A = euclidean(coords[1], coords[5])
    B = euclidean(coords[2], coords[4])
    C = euclidean(coords[0], coords[3])
    return (A + B) / (2.0 * C)

def get_gaze_direction(landmarks, eye_idx, iris_idx, w, h):
    eye_coords = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in eye_idx]
    iris_coords = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in iris_idx]
    eye_left = eye_coords[0][0]
    eye_right = eye_coords[3][0]
    iris_center_x = sum([p[0] for p in iris_coords]) // len(iris_coords)
    position_ratio = (iris_center_x - eye_left) / (eye_right - eye_left + 1e-6)
    return position_ratio

def get_head_pose(landmarks, w, h):
    image_points = np.array([
        (landmarks[i].x * w, landmarks[i].y * h) for i in FACE_2D_IDXS
    ], dtype=np.float64)
    # Camera internals
    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype=np.float64)
    dist_coeffs = np.zeros((4, 1))
    success, rotation_vector, translation_vector = cv2.solvePnP(
        FACE_3D_POINTS, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
    )
    if not success:
        return None
    # Convert rotation vector to Euler angles
    rmat, _ = cv2.Rodrigues(rotation_vector)
    proj_matrix = np.hstack((rmat, translation_vector))
    _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(proj_matrix)
    yaw = euler_angles[1][0]  # Yaw (left/right)
    return yaw

# State variables
blink_count = 0
blink_times = []
gaze_away_times = []
blink_state = False
gaze_warning_active = False  # To avoid repeated beeps

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    suspicious_warning = False
    drowsy_warning = False
    head_turn_warning = False
    face_not_found = False

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        # Blink detection
        left_ear = eye_aspect_ratio(landmarks, LEFT_EYE, w, h)
        right_ear = eye_aspect_ratio(landmarks, RIGHT_EYE, w, h)
        avg_ear = (left_ear + right_ear) / 2.0

        if avg_ear < EAR_THRESHOLD:
            if not blink_state:
                blink_state = True
        else:
            if blink_state:
                blink_state = False
                blink_count += 1
                blink_times.append(time.time())
                winsound.Beep(1000, 150)

        # Gaze detection
        gaze_ratio = get_gaze_direction(landmarks, LEFT_EYE, LEFT_IRIS, w, h)
        if gaze_ratio < GAZE_LEFT_LIMIT or gaze_ratio > GAZE_RIGHT_LIMIT:
            gaze_away_times.append(time.time())

        # Head pose detection
        yaw = get_head_pose(landmarks, w, h)
        if yaw is not None and abs(yaw) > 25:  # threshold in degrees
            head_turn_warning = True
    else:
        face_not_found = True

    # Clean old timestamps
    now = time.time()
    blink_times = [t for t in blink_times if now - t < 60]
    gaze_away_times = [t for t in gaze_away_times if now - t < 60]
    blink_rate = len(blink_times)
    gaze_away_count = len(gaze_away_times)

    # Display text
    cv2.putText(frame, f"Blinks: {blink_count}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv2.putText(frame, f"Rate: {blink_rate}/min", (30, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100, 255, 255), 2)

    if face_not_found:
        # Show face not detected message at the top center, suppress all other warnings
        text = "Face not detected!"
        textsize = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)[0]
        textX = (frame.shape[1] - textsize[0]) // 2
        cv2.putText(frame, text, (textX, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        gaze_warning_active = False  # Reset gaze warning state
    else:
        y_offset = 110
        if blink_rate > DROWSINESS_BLINK_THRESHOLD:
            drowsy_warning = True
            cv2.putText(frame, "WARNING: Drowsiness Detected!", (30, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
            y_offset += 40

        if gaze_away_count > SUSPICIOUS_GAZE_LIMIT:
            suspicious_warning = True
            cv2.putText(frame, "Please look on screen", (30, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            if not gaze_warning_active:
                winsound.Beep(1200, 200)
                gaze_warning_active = True
            y_offset += 40
        else:
            gaze_warning_active = False  # Reset when not in warning

        if head_turn_warning:
            cv2.putText(frame, "Suspicious head turn detected!", (30, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Exam_Checker", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 